In [5]:
import json
path_to_json = '../public/urban_district.hbjson'
output_path = path_to_json.replace(".hbjson", ".glb")
hbjson = json.load(open(path_to_json))

In [6]:
import json
import numpy as np
import trimesh

# Define OpenStudio color palette for different building elements
OPENSTUDIO_COLORS = {
    "walls": [0.8, 0.5, 0.2],
    "interior_walls": [0.6, 0.6, 0.6],
    "roofs": [0.5, 0.1, 0.1],
    "ceilings": [0.7, 0.7, 0.7],
    "exterior_floors": [0.4, 0.3, 0.2],
    "interior_floors": [0.6, 0.4, 0.3],
    "air_walls": [0.9, 0.9, 0.9],
    "apertures": [0.1, 0.5, 0.9],
    "interior_apertures": [0.3, 0.7, 1.0],
    "doors": [0.5, 0.3, 0.2],
    "interior_doors": [0.5, 0.4, 0.3],
    "outdoor_shades": [0.3, 0.3, 0.3],
    "indoor_shades": [0.5, 0.5, 0.5],
    "shade_meshes": [0.2, 0.2, 0.2],
}

# Mapping face_type to categories
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",  # Default, updated below based on boundary conditions
}

path_to_json = '../public/urban_district.hbjson'
output_path = path_to_json.replace(".hbjson", ".glb")
hbjson = json.load(open(path_to_json))

# Dictionary to store categorized geometry elements
categorized_meshes = {category: [] for category in OPENSTUDIO_COLORS.keys()}

# Rotation matrix to correct orientation (90-degree rotation along X-axis)
rotation_matrix = trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])

# Iterate through rooms and extract geometry
for room in hbjson.get("rooms", []):
    for face in room.get("faces", []):
        geometry = face.get("geometry", {})
        boundary = geometry.get("boundary", [])

        if len(boundary) < 3:
            continue  # Skip invalid faces

        # Convert boundary to numpy array
        vertices = np.array(boundary, dtype=np.float32)
        faces = [[0, i, i + 1] for i in range(1, len(vertices) - 1)]

        # Determine category based on face_type
        face_type = face.get("face_type", "Wall")
        category = FACE_TYPE_MAPPING.get(face_type, "walls")

        # Adjust floors based on boundary condition
        boundary_condition = face.get("boundary_condition", {}).get("type", "")
        if face_type == "Floor":
            category = "exterior_floors" if boundary_condition == "Ground" else "interior_floors"

        if category in categorized_meshes:
            mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
            mesh.apply_transform(rotation_matrix)  # Apply rotation fix
            mesh.visual.vertex_colors = OPENSTUDIO_COLORS.get(category, [1.0, 1.0, 1.0])
            categorized_meshes[category].append(mesh)

        # ---- Add Apertures (Windows, Skylights) ----
        for aperture in face.get("apertures", []):
            aperture_geometry = aperture.get("geometry", {})
            aperture_boundary = aperture_geometry.get("boundary", [])

            if len(aperture_boundary) < 3:
                continue  # Skip invalid apertures

            aperture_vertices = np.array(aperture_boundary, dtype=np.float32)
            aperture_faces = [[0, i, i + 1] for i in range(1, len(aperture_vertices) - 1)]

            # Classify apertures
            aperture_category = "apertures"
            if face_type == "Wall":
                aperture_category = "apertures"  # Windows
            elif face_type == "RoofCeiling":
                aperture_category = "interior_apertures"  # Skylights

            if aperture_category in categorized_meshes:
                aperture_mesh = trimesh.Trimesh(vertices=aperture_vertices, faces=aperture_faces)
                aperture_mesh.apply_transform(rotation_matrix)  # Apply rotation fix
                aperture_mesh.visual.vertex_colors = OPENSTUDIO_COLORS.get(aperture_category, [1.0, 1.0, 1.0])
                categorized_meshes[aperture_category].append(aperture_mesh)

# Combine meshes into a single model
final_meshes = []
for category, meshes in categorized_meshes.items():
    if meshes:
        final_meshes.append(trimesh.util.concatenate(meshes))

if final_meshes:
    combined_mesh = trimesh.util.concatenate(final_meshes)
    combined_mesh.export(output_path)
    print(f"GLB file saved: {output_path}")
else:
    print("No valid geometry found in HBJSON.")

GLB file saved: ../public/urban_district.glb


In [42]:
import json
import numpy as np
import trimesh

# Define OpenStudio color palette for different building elements
OPENSTUDIO_COLORS = {
    "walls": [0.8, 0.5, 0.2],
    "apertures": [0.1, 0.5, 0.9],  # Windows
}

# Mapping face_type to categories
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",
}

#path_to_json = "../public/demo.hbjson"
path_to_json = "../public/urban_district.hbjson"
output_path = path_to_json.replace(".HBJSON", ".glb")
hbjson = json.load(open(path_to_json))

# Dictionary to store categorized geometry elements
categorized_meshes = {category: [] for category in OPENSTUDIO_COLORS.keys()}

# Rotation matrix to correct orientation (90-degree rotation along X-axis)
rotation_matrix = trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])


def create_wall_with_holes(outer_boundary, holes):
    """
    Creates a wall mesh with window cutouts using indexed faces.
    """
    # Convert to numpy arrays
    outer_boundary = np.array(outer_boundary, dtype=np.float32)
    hole_boundaries = [np.array(hole, dtype=np.float32) for hole in holes]

    # Combine outer boundary with holes
    all_vertices = np.vstack([outer_boundary] + hole_boundaries)
    
    # Create face indices: outer boundary first
    face_indices = [[i, (i + 1) % len(outer_boundary), (i + 2) % len(outer_boundary)]
                    for i in range(len(outer_boundary) - 2)]

    # Add inner holes as separate loops
    hole_start_idx = len(outer_boundary)
    for hole in hole_boundaries:
        hole_indices = [[hole_start_idx + i, hole_start_idx + (i + 1) % len(hole), hole_start_idx + (i + 2) % len(hole)]
                        for i in range(len(hole) - 2)]
        face_indices.extend(hole_indices)
        hole_start_idx += len(hole)

    # Create Trimesh object
    wall_mesh = trimesh.Trimesh(vertices=all_vertices, faces=np.array(face_indices, dtype=np.int32))
    
    return wall_mesh


# Iterate through rooms and extract geometry
for room in hbjson.get("rooms", []):
    for face in room.get("faces", []):
        geometry = face.get("geometry", {})
        boundary = geometry.get("boundary", [])

        if len(boundary) < 3:
            continue  # Skip invalid faces

        wall_vertices = np.array(boundary, dtype=np.float32)

        # Collect windows (apertures) as holes
        holes = []
        for aperture in face.get("apertures", []):
            aperture_geometry = aperture.get("geometry", {})
            aperture_boundary = aperture_geometry.get("boundary", [])

            if len(aperture_boundary) >= 3:
                holes.append(np.array(aperture_boundary, dtype=np.float32))

        # Create the proper face with holes
        wall_mesh = create_wall_with_holes(wall_vertices, holes)
        wall_mesh.apply_transform(rotation_matrix)  # Apply rotation fix

        # Determine category based on face_type
        face_type = face.get("face_type", "Wall")
        category = FACE_TYPE_MAPPING.get(face_type, "walls")

        if category in categorized_meshes:
            wall_mesh.visual.vertex_colors = OPENSTUDIO_COLORS.get(category, [1.0, 1.0, 1.0])
            categorized_meshes[category].append(wall_mesh)

# Combine meshes into a single model
final_meshes = [trimesh.util.concatenate(meshes) for meshes in categorized_meshes.values() if meshes]

if final_meshes:
    combined_mesh = trimesh.util.concatenate(final_meshes)
    combined_mesh.export(output_path, file_type='glb')  # ✅ Export as GLB
    print(f"GLB file saved: {output_path}")
else:
    print("No valid geometry found in HBJSON.")


GLB file saved: ../public/urban_district.hbjson


In [4]:
import json
import numpy as np
import trimesh
from pygltflib import GLTF2, Scene, Node, Mesh, Buffer, BufferView, Accessor, Asset, Primitive, Material

# Define OpenStudio color palette for different building elements
OPENSTUDIO_COLORS = {
    "walls": [0.8, 0.5, 0.2, 1.0],
    "interior_walls": [0.6, 0.6, 0.6, 1.0],
    "roofs": [0.5, 0.1, 0.1, 1.0],
    "ceilings": [0.7, 0.7, 0.7, 1.0],
    "exterior_floors": [0.4, 0.3, 0.2, 1.0],
    "interior_floors": [0.6, 0.4, 0.3, 1.0],
    "air_walls": [0.9, 0.9, 0.9, 1.0],
    "apertures": [0.1, 0.5, 0.9, 1.0],
    "interior_apertures": [0.3, 0.7, 1.0, 1.0],
    "doors": [0.5, 0.3, 0.2, 1.0],
    "interior_doors": [0.5, 0.4, 0.3, 1.0],
    "outdoor_shades": [0.3, 0.3, 0.3, 1.0],
    "indoor_shades": [0.5, 0.5, 0.5, 1.0],
    "shade_meshes": [0.2, 0.2, 0.2, 1.0],
}

# Mapping face_type to categories
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",  # Default, updated below based on boundary conditions
}

path_to_json = '../public/urban_district.hbjson'
output_path = path_to_json.replace(".hbjson", ".glb")
hbjson = json.load(open(path_to_json))

# Dictionary to store categorized geometry elements
categorized_meshes = {category: [] for category in OPENSTUDIO_COLORS.keys()}

# Rotation matrix to correct orientation (90-degree rotation along X-axis)
rotation_matrix = trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])

# Iterate through rooms and extract geometry
for room in hbjson.get("rooms", []):
    for face in room.get("faces", []):
        geometry = face.get("geometry", {})
        boundary = geometry.get("boundary", [])

        if len(boundary) < 3:
            continue  # Skip invalid faces

        # Convert boundary to numpy array
        vertices = np.array(boundary, dtype=np.float32)
        faces = [[0, i, i + 1] for i in range(1, len(vertices) - 1)]

        # Determine category based on face_type
        face_type = face.get("face_type", "Wall")
        category = FACE_TYPE_MAPPING.get(face_type, "walls")

        # Adjust floors based on boundary condition
        boundary_condition = face.get("boundary_condition", {}).get("type", "")
        if face_type == "Floor":
            category = "exterior_floors" if boundary_condition == "Ground" else "interior_floors"

        if category in categorized_meshes:
            mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
            mesh.apply_transform(rotation_matrix)  # Apply rotation fix
            mesh.visual.vertex_colors = OPENSTUDIO_COLORS.get(category, [1.0, 1.0, 1.0])
            categorized_meshes[category].append(mesh)

        # ---- Add Apertures (Windows, Skylights) ----
        for aperture in face.get("apertures", []):
            aperture_geometry = aperture.get("geometry", {})
            aperture_boundary = aperture_geometry.get("boundary", [])

            if len(aperture_boundary) < 3:
                continue  # Skip invalid apertures

            aperture_vertices = np.array(aperture_boundary, dtype=np.float32)
            aperture_faces = [[0, i, i + 1] for i in range(1, len(aperture_vertices) - 1)]

            # Classify apertures
            aperture_category = "apertures"
            if face_type == "Wall":
                aperture_category = "apertures"  # Windows
            elif face_type == "RoofCeiling":
                aperture_category = "interior_apertures"  # Skylights

            if aperture_category in categorized_meshes:
                aperture_mesh = trimesh.Trimesh(vertices=aperture_vertices, faces=aperture_faces)
                aperture_mesh.apply_transform(rotation_matrix)  # Apply rotation fix
                aperture_mesh.visual.vertex_colors = OPENSTUDIO_COLORS.get(aperture_category, [1.0, 1.0, 1.0])
                categorized_meshes[aperture_category].append(aperture_mesh)

# Combine meshes into a single model
final_meshes = []
for category, meshes in categorized_meshes.items():
    if meshes:
        final_meshes.append(trimesh.util.concatenate(meshes))

if final_meshes:
    combined_mesh = trimesh.util.concatenate(final_meshes)
    vertices = combined_mesh.vertices.flatten().tolist()
    indices = combined_mesh.faces.flatten().tolist()
    colors = combined_mesh.visual.vertex_colors.flatten().tolist()

    # Create GLTF2 object
    gltf = GLTF2(
        asset=Asset(version="2.0"),
        scenes=[Scene(nodes=[0])],
        nodes=[Node(mesh=0)],
        meshes=[
            Mesh(primitives=[
                Primitive(
                    attributes={
                        "POSITION": 0,
                        "COLOR_0": 1
                    },
                    indices=2,
                    material=0
                )
            ])
        ],
        materials=[
            Material(
                pbrMetallicRoughness=None
            )
        ],
        buffers=[
            Buffer(
                byteLength=len(vertices) * 4 + len(indices) * 4 + len(colors) * 4
            )
        ],
        bufferViews=[
            BufferView(
                buffer=0,
                byteOffset=0,
                byteLength=len(vertices) * 4,
                target=34962  # ARRAY_BUFFER
            ),
            BufferView(
                buffer=0,
                byteOffset=len(vertices) * 4,
                byteLength=len(colors) * 4,
                target=34962  # ARRAY_BUFFER
            ),
            BufferView(
                buffer=0,
                byteOffset=len(vertices) * 4 + len(colors) * 4,
                byteLength=len(indices) * 4,
                target=34963  # ELEMENT_ARRAY_BUFFER
            )
        ],
        accessors=[
            Accessor(
                bufferView=0,
                byteOffset=0,
                componentType=5126,  # FLOAT
                count=len(vertices) // 3,
                type="VEC3"
            ),
            Accessor(
                bufferView=1,
                byteOffset=0,
                componentType=5126,  # FLOAT
                count=len(colors) // 4,
                type="VEC4"
            ),
            Accessor(
                bufferView=2,
                byteOffset=0,
                componentType=5123,  # UNSIGNED_SHORT
                count=len(indices),
                type="SCALAR"
            )
        ]
    )

    # Save GLB file
    gltf.save(output_path)
    print(f"GLB file saved: {output_path}")
else:
    print("No valid geometry found in HBJSON.")

TypeError: 'NoneType' object is not subscriptable